# RAG 
## 2. 로컬에 저장(chroma 파일을 만들고 거기에 저장)
1. 문서 내용 읽기
    - https://python.langchain.com/v0.2/docs/integrations/document_loaders/
1. 문서 분할(쪼개기)
    - 문서 분할하지 않으면?
        - 토큰 수 초과로 답변을 생성하지 않을 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 - > 답변 늦어짐
1. 임베딩 - 벡터 데이터베이스에 저장
1. 질문이 있으면 벡터 데이터베이스에 유사도 검색(코사인 유사도)
1. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [7]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from dotenv import load_dotenv


## 1, 2. 문서 Microsoft Word 파일 읽고 분할 한번에 
loader = Docx2txtLoader("law_1.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

data_list = loader.load_and_split(text_splitter=text_splitter)


# 3. 임베딩 -> 벡터 데이터베이스에 저장 
## 환경변수 읽어오기
load_dotenv()

## 3-1. 임베딩 
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)


In [ ]:

## 3-2 벡터 데이터베이스에 저장 
## [방법1] inmemory 보통은 인메모리에 저장하지 않음. 
# database = Chroma.from_documents(
#     documents=data_list,
#     embedding=embedding,
# )

## [방법2] 로컬에 저장(chroma 파일을 만들고 거기에 저장) 
# 한번 실행하면 생기기 때문에 두번 세번 할 필요가 없음.
# database = Chroma.from_documents(
#     documents=data_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

# 2.1 두번째부터는 로컬에 저장된 임베딩 데이터 가져오기
database = Chroma(
    embedding_function=embedding,        
    persist_directory='./chroma',                  
    collection_name='chroma-law',
)

database


In [13]:


## 4. 사용자 질문 + 벡터 데이터베이스에서 유사도 검색
# "전세사기피해자 금융지원에 대해 설명해주세요."
query = "전세사기피해자로 인정받기 위한 조건은? " 


## 벡터 데이터베이스에서 유사도 검색(기본이 4개씩)
retrieved_docs = database.similarity_search(query=query, k=3)


## 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 
## 5-1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기피해 법률 전문가입니다
-[context]를 참고해 사용자의 질문에 답변해주세요. 

[context]
{retrieved_docs} 

Question : {query}

'''

formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,
)

## 5-2 LLM 모델 생성, ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model_name='gpt-4.1-nano'
)

aimessage = llm.invoke(formatted_prompt)
aimessage

AIMessage(content='전세사기피해자로 인정받기 위한 조건은 다음과 같습니다:\n\n1. 임차인으로서 주택임대차보호법 제3조에 따라 주택의 인수와 주민등록을 마치고, 임대차계약서에 대한 확정일자를 갖추거나, 임차권등기를 마친 상태여야 합니다(임차권등기 또는 전세권 설정이 포함됨).\n\n2. 임차보증금이 5억 원 이하이어야 하며, 필요 시 시·도별 피해 상황에 따라 상한액이 2억 원으로 조정될 수 있습니다.\n\n3. 임대인의 파산, 회생절차 개시, 경매 또는 공매절차 개시 등으로 인해 임차보증금 반환이 어려운 피해를 입었거나 예상되는 상태여야 합니다. 또한, 다수의 임차인에게 채권반환이 어려운 경우도 포함됩니다.\n\n4. 임대인 또는 관련자에 대한 수사 개시, 기망 행위, 임차보증금을 반환할 능력이 없는 자에게 주택을 양도하거나 임차보증금을 반환할 능력 없이 다수의 주택을 취득 또는 임대하는 등의 임대인이 임차보증금 반환 채무를 이행하지 아니할 의사가 의심될 만한 상당한 이유가 있어야 합니다.\n\n추가로, 일부 경우(임차인 임차보증금 반환을 위한 보증보험 가입 또는 임차보증금 전액이 최우선변제 가능한 경우 등)에는 전세사기피해자로 인정받지 않을 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 352, 'prompt_tokens': 3137, 'total_tokens': 3489, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14'